# Faire des statistiques

## Données

Mariannig Le Béchec - Aline Bouchard - Philippe Charrier - Claire Denecker - Gabriel Gallezot - Stéphanie Rennes

- Rapport : https://hal.archives-ouvertes.fr/hal-03545512v1
- Données : https://zenodo.org/record/5827206


In [2]:
import pandas as pd
df = pd.read_csv("https://zenodo.org/records/5827206/files/SOSP_Export_base%20de%20donn%C3%A9es%20diffusable.csv")

Deux variables :
- année de naissance
- présentation dans les réseaux sociaux : Valorisation_recherche_reseaux_sociaux_num_generalistes

Produire les différentes catégories

In [ ]:
df["annee_num"] = df["annee_premiere_publi"]
df["annee_cat"] = pd.cut(df["annee_premiere_publi"], 5)

df["reseauxsociaux_cat"] = df["Valorisation_recherche_reseaux_sociaux_num_generalistes"]
reco = {'parfois':2, 'souvent':3, 'jamais':0, 'rarement':1, 'toujours':4}
df["reseauxsociaux_num"] = df["reseauxsociaux_cat"].replace(reco)

## Univarié

Indicateurs

Distribution

recodage

## Bivarié

Quanti/quanti .corr

In [ ]:
from scipy.stats import pearsonr

PearsonRResult(statistic=0.19285061858630462, pvalue=1.3890221497996923e-10)

Quanti/quali

.groupby

In [ ]:
from scipy.stats import f_oneway

F_onewayResult(statistic=10.73068474537501, pvalue=1.5652446052537652e-08)

Possibilité d'utiliser d'autres bibliothèques, comme pinguin https://pingouin-stats.org/build/html/index.html

In [60]:
#pip install pingouin

Source             SS    DF          MS         F    p-unc       np2
0  reseaux_n     961.069233     4  240.267308  1.866877  0.11399  0.006842
1     Within  139510.930767  1084  128.700121       NaN      NaN       NaN

Quali/Quali

pd.crosstab

In [39]:
from scipy.stats import chi2_contingency

## Modèles

#### Régression logistique

Quels sont les prédicteurs du fait de diffuser sa recherche sur les médias sociaux

Création d'une variable 0 (pas du tout ou rarement) et 1 sinon

In [83]:
reco = {'rarement':0, 'parfois':1, 'jamais':0, 'souvent':0, 'toujours':0}
df["reseaux_b"] = df["Valorisation_recherche_reseaux_sociaux_num_generalistes"].replace(reco)
df["reseaux_b"].value_counts()

reseaux_b
0    983
1    106
Name: count, dtype: int64

In [112]:
df["sexe_r"] = df["sexe"].replace({"je ne souhaite pas répondre":None})

Avec Scikit-learn

In [75]:
from sklearn.linear_model import LogisticRegression

In [113]:
X = pd.get_dummies(df[["annee_premiere_publi","sexe_r"]], drop_first=True)
y = df["reseaux_b"]

In [114]:
modele = LogisticRegression()
modele.fit(X, y)
modele.coef_

array([[-1.11056683e-03, -1.86439035e-07]])

Avec Statsmodels

In [118]:
import statsmodels.api as sm
from patsy import dmatrices

In [119]:
y,X = dmatrices('reseaux_b ~ sexe_r + annee_n', 
                data = df, 
                return_type = 'dataframe')

In [126]:
modele = sm.Logit(y,X)
resultat = modele.fit()
resultat.summary()

Optimization terminated successfully.
         Current function value: 0.316656
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              reseaux_b   No. Observations:                 1045
Model:                          Logit   Df Residuals:                     1042
Method:                           MLE   Df Model:                            2
Date:                Fri, 03 May 2024   Pseudo R-squ.:                0.009814
Time:                        15:39:03   Log-Likelihood:                -330.91
converged:                       True   LL-Null:                       -334.19
Covariance Type:            nonrobust   LLR p-value:                   0.03764
=======================================================================================
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept             -48.9593     19.256     -2.543      0.011     -86.700     -11.219
sexe_r[T.une femme]     0.1321      0.209      0.631      0.528      -0.278       0.542
annee_n                 0.0233      0.010      2.425      0.015       0.004       0.042
=======================================================================================
"""

Avec la version toute intégrée

In [129]:
#pip install pyshs

In [130]:
import pyshs

In [131]:
pyshs.regression_logistique(df, "reseaux_b", ["sexe_r", "annee_n"])

OR       p            IC 95%
Variable   Modalité                                 
.Intercept             0.0  0.011*  0.00 [0.00-0.00]
annee_n    numérique  1.02  0.015*  1.02 [1.00-1.04]
sexe_r     un homme    ref                          
           une femme  1.14   0.528  1.14 [0.76-1.72]